In [1]:
%load_ext autoreload
%autoreload 2

# default 

In [5]:
#!pip install job-offer-classifier --upgrade

In [3]:
from os import path,listdir
from shutil import rmtree
import pandas as pd
import numpy as np
import job_offer_classifier
from job_offer_classifier.pipeline_classifier import Pipeline
from job_offer_classifier.validations import KFoldPipe
from job_offer_classifier.pipeline_classifier import plot_confusion_matrix

In [4]:
job_offer_classifier.__version__

'0.0.3'

In [ ]:
# Shutdown TF info logging
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Load the dataset

The dataset consists of mock emails by the Intekglobal employees. 

In [ ]:
df = pd.read_csv('../data/interim/mocks.csv')

The original data consisted of folders with the information provided by each employee. The preprocessing of can be found below in the **Preprocess the Mocks** section.

**The dataframe has the following structure:**

In [ ]:
df.sample(2,random_state=200)

In [ ]:
print(f'Total number of mocks: {df.shape[0]}')

In [ ]:
print(f'Number of mocks per mocker: \n{df.mocker.value_counts()}')

In [ ]:
print(f'Number of different sentiments: \n{df.sentiment.value_counts()}')

In [ ]:
ex = df.sample().iloc[0]
print(ex.payload)
print(f'\nsentiment: {ex.sentiment}')

## Predict Sentiments

### Approach 1

Initiate the `Pipeline` passing the estimator directory name, the dataset file arguments. Also, set the fraction of the training data equal to zero.

In [ ]:
estimator_dir ='../models/estimator'
assert path.isdir(estimator_dir)

In [ ]:
pl = Pipeline(
    src_file='../data/interim/mocks.csv',
    estimator_dir=estimator_dir,
    frac=0.0
)

The estimator has been trained in the **Load and Train the Estimator** section (see below). Next, load the estimator and the TF functions:

In [ ]:
pl.load_estimator()
pl.input_fns()

The resulting functions are stored in the `input` dictionary.  The training dataframe is empty and the test data has all the mock samples:

In [ ]:
assert pl.dfs['train'].size == 0
assert pl.dfs['test'].shape == df.shape

 [To prevent an evaluation over an empty set](https://github.com/kikejimenez/job_offer_classifier/issues/7), eliminate the TF function corresponding to the training set:

In [ ]:
del pl.input['predict']['train']

In [ ]:
pl.evaluate()

In [ ]:
pl.evaluation

In [ ]:
pl.plot_confusion_matrix('test')

Since the dataset has been passed to the `Pipeline` instance with zero fraction of train samples, the whole data is contained in the 'test' part:

### Second Approach

In [ ]:
pl = Pipeline(estimator_dir = estimator_dir)
pl.load_estimator()
path.isdir(pl.estimator.model_dir)
#pl.dfs = {'test':df}
#pl.input_fns()

In [ ]:
pl.evaluate()

In [ ]:
pl = Pipeline(estimator_dir = estimator_dir)
pl.load_estimator()

In [ ]:
df['prediction'] = pl.predict(df)

In [ ]:
df.prediction.value_counts()

In [ ]:
def confusion_matrix(df):
    TF  = df.prediction.value_counts
    